In [1]:
import preprocessing as pp
import load_files as lf
import nlp_algorithms as nlp

import pandas as pd
import numpy as np

from sentence_transformers import SentenceTransformer

In [2]:
pd.set_option('display.max_colwidth', -1)
## dfSourceV2 = pd.read_csv('../Dados/v2_semeval-2022_task8_train-data_batch.csv')
train_v1 = pd.read_csv('dados/train v0.1.csv')

train_v1_enen = train_v1[(train_v1['url1_lang'] == 'en') & (train_v1['url2_lang'] == 'en')]

train_v1_enen.head()

C:\Users\lucas\AppData\Local\Temp/ipykernel_11328/2099918564.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,url1_lang,url2_lang,pair_id,link1,link2,ia_link1,ia_link2,Geography,Entities,Time,Narrative,Overall,Style,Tone
0,en,en,1484084337_1484110209,https://www.washingtonpost.com/local/virginia-man-arrested-in-fatal-dui-crash-in-west-virginia/2020/01/01/740fbc7a-2cbe-11ea-bffe-020c88b3f120_story.html,https://www.washingtonpost.com/world/the_americas/haitis-leader-marks-independence-day-amid-security-concerns/2020/01/01/dc4033a4-2cc5-11ea-bffe-020c88b3f120_story.html,https://web.archive.org/web/www.washingtonpost.com/local/virginia-man-arrested-in-fatal-dui-crash-in-west-virginia/2020/01/01/740fbc7a-2cbe-11ea-bffe-020c88b3f120_story.html,https://web.archive.org/web/www.washingtonpost.com/world/the_americas/haitis-leader-marks-independence-day-amid-security-concerns/2020/01/01/dc4033a4-2cc5-11ea-bffe-020c88b3f120_story.html,4.0,4.000000,1.000000,4.000000,4.000000,1.666667,2.000000
1,en,en,1484396422_1483924666,https://www.stlucianewsonline.com/guyana-three-injured-after-car-crashes-into-utility-pole/,https://www.thestar.com/news/world/europe/2020/01/01/fire-kills-animals-at-zoo-in-western-germany.html,https://web.archive.org/web/www.stlucianewsonline.com/guyana-three-injured-after-car-crashes-into-utility-pole/,https://web.archive.org/web/www.thestar.com/news/world/europe/2020/01/01/fire-kills-animals-at-zoo-in-western-germany.html,4.0,4.000000,1.000000,4.000000,3.666667,1.666667,1.333333
2,en,en,1484698254_1483758694,https://www.teaparty.org/trump-brings-in-2020-at-mar-a-lago-were-going-to-have-a-great-year-423052/,https://www.timesofisrael.com/trump-says-he-does-not-expect-war-with-iran-likes-peace/,https://web.archive.org/web/www.teaparty.org/trump-brings-in-2020-at-mar-a-lago-were-going-to-have-a-great-year-423052/,https://web.archive.org/web/www.timesofisrael.com/trump-says-he-does-not-expect-war-with-iran-likes-peace/,1.0,2.000000,1.000000,2.333333,2.333333,1.000000,1.333333
3,en,en,1576314516_1576455088,https://gadgets.ndtv.com/apps/news/zomato-uber-eats-business-acquisition-india-all-stock-deal-2167155,https://gadgets.ndtv.com/internet/news/indian-online-food-delivery-market-to-hit-usd-8-billion-by-2020-google-bcg-report-2171043,https://web.archive.org/web/gadgets.ndtv.com/apps/news/zomato-uber-eats-business-acquisition-india-all-stock-deal-2167155,https://web.archive.org/web/gadgets.ndtv.com/internet/news/indian-online-food-delivery-market-to-hit-usd-8-billion-by-2020-google-bcg-report-2171043,1.0,2.333333,2.666667,1.666667,2.000000,1.666667,1.666667
4,en,en,1484036253_1483894099,https://news.yahoo.com/india-approves-third-moon-mission-085759387.html,https://www.channelnewsasia.com/news/asia/india-targets-new-moon-mission-in-2020-12225344,https://web.archive.org/web/news.yahoo.com/india-approves-third-moon-mission-085759387.html,https://web.archive.org/web/www.channelnewsasia.com/news/asia/india-targets-new-moon-mission-in-2020-12225344,1.0,1.250000,1.000000,1.250000,1.250000,1.000000,1.000000


In [ ]:
train_v2 = pd.read_csv('dados/train v0.2.csv')

train_v2_enen = train_v2[(train_v2['url1_lang'] == 'en') & (train_v2['url2_lang'] == 'en')]

train_v2_enen.head()

In [ ]:
test = pd.read_csv('dados/eval/final_evaluation_data.csv')

test_enen = test[(test['url1_lang'] == 'en') & (test['url2_lang'] == 'en')]

test_enen.head()

In [ ]:
train_v1_enen.shape

In [ ]:
test_enen.shape

In [3]:
data_path = 'dados/train v0.1/'

lista_docs = []
lista_error = []
lista_vazio = []
values = train_v1_enen[['pair_id', 'Overall']]

for index, values in values.iterrows():
    
    try:
        
        json_pair = lf.get_json_document_pair(data_path, values['pair_id'])
        text_doc1 = json_pair[0]['text']
        text_doc2 = json_pair[1]['text']
        
        if ( len(text_doc1) > 0 ) and ( len(text_doc2) > 0 ):
            lista_docs.append((values['pair_id'], text_doc1, text_doc2, values['Overall']))
        else:
            lista_vazio.append(values['pair_id'])
    
    except:
        lista_error.append(values['pair_id'])

In [4]:
df_text = pd.DataFrame(lista_docs,  columns=['pair_id', 'doc1', 'doc2', 'Overall'])

In [5]:
df_text.shape

(1357, 4)

## Pre-processamento

In [6]:
'''pre_processing_list = [
    {},
    {"no_url": True},
    {"basic_processing": True},
    {"no_url": True, "basic_processing": True},
    {"no_url": True, "basic_processing": True, "no_stopwords": True}
    ]'''

pre_processing_list = [
    {},
    {"no_url": True, "basic_processing": True}
    ]


df_pp = pp.pre_process_all(df_text, pre_processing_list)

# Algoritmos

In [7]:
df_results = df_pp[["pair_id", "Overall"]]
df_results.head(1)

,pair_id,Overall
0,1484084337_1484110209,4.0


## BERT

In [ ]:
pre_processing_list

In [ ]:
'''## Funcoes do encode pro fine tune
import tensorflow as tf
import tensorflow_addons as tfa

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy

from transformers import TFBertForSequenceClassification
from transformers import BertTokenizer

# can be up to 512 for BERT
MAX_LENGTH = 256
BATCH_SIZE = 64
SEED = 42
from transformers import BertTokenizer

tokenizer = SentenceTransformer('sentence-transformers/bert-base-nli-mean-tokens')
#BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case = False)

def map_example_to_dict(input_ids, attention_masks, token_type_ids, label):
  return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }, label

def convert_example_to_feature(titulo_1, titulo_2):
    return tokenizer.encode_plus(titulo_1, titulo_2,
                                 add_special_tokens = True, # adiciona [CLS], [SEP]
                                 max_length = MAX_LENGTH, # comprimento máximo do texto de entrada
                                 padding = 'max_length', # adiciona [PAD] até o tam_max (MAX_LENGTH)
                                 truncation = True, # padrão = 'longest_first'
                                 return_attention_mask = True, # adiciona máscara de atenção para não focar nos tokens do pad
                                )
                        
def encode_examples(df_titulos, labels, limit = -1):
    
    # prepare list, so that we can build up final TensorFlow dataset from slices.
    input_ids_list = []
    token_type_ids_list = []
    attention_mask_list = []
    label_list = []
    
    if (limit > 0):
        ds = ds.take(limit)
    
    # for review, label in tfds.as_numpy(ds):
    for titulo_1, titulo_2, label in zip(df_titulos["titulo_1"], df_titulos["titulo_2"], labels):
        
        bert_input = convert_example_to_feature(titulo_1, titulo_2)
        input_ids_list.append(bert_input['input_ids'])
        token_type_ids_list.append(bert_input['token_type_ids'])
        attention_mask_list.append(bert_input['attention_mask'])
        label_list.append([label])
        
    return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, label_list)).map(map_example_to_dict)

def get_bert_data(X_train, y_train, X_valid, y_valid):
    
    # train dataset
    ds_train = encode_examples(X_train, y_train).batch(BATCH_SIZE)

    #validation dataset
    ds_valid = encode_examples(X_valid, y_valid).batch(BATCH_SIZE)

    return ds_train, ds_valid

import pandas as pd
import numpy as np
import time

from sklearn.metrics.pairwise import cosine_similarity

from scipy import spatial
from sklearn.model_selection import train_test_split

###Bert###
def get_bert(model, doc1, doc2):
    
    data = [doc1, doc2]
    sentence_embeddings = model.encode(data)

    infer1 = sentence_embeddings[0]
    infer2 = sentence_embeddings[1]
    
    cos_similarity = 1 - spatial.distance.cosine(infer1, infer2) #de 0 a 1
    
    return cos_similarity



def apply_bert(df, len_pipeline, model, model_name, fine_tune = False):

    time_list = []

    df_bert = pd.DataFrame()
    for index in range(len_pipeline):
        if fine_tune == True:
            X = df[f'doc1_pipeline{index}']
            y = df['Overall']
            X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2, random_state = SEED, stratify = y)

            learning_rate = 2e-5
            number_of_epochs = 3
            ds_train, ds_valid, ds_test = get_bert_data(X_train, y_train, X_valid, y_valid)
            
            # model initialization
            #model = TFBertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased', from_pt = True)

            # choosing Adam optimizer
            optimizer = Adam(learning_rate=learning_rate, epsilon = 1e-08)
            loss = SparseCategoricalCrossentropy(from_logits = True)
            metric_acc = SparseCategoricalAccuracy('accuracy')
            model.compile(optimizer = optimizer, loss = loss, metrics = [metric_acc])

            #Training model
            bert_history = model.fit(ds_train, epochs = number_of_epochs, validation_data = ds_valid)
        start_time = time.time()
        df_bert[f'bert_{model_name}{index}'] = df.apply(lambda row: get_bert(model, " ".join(row[f'doc1_pipeline{index}']), " ".join(row[f'doc2_pipeline{index}'])), axis=1)
        time_list.append((f'bert_{model_name}{index}', time.time()-start_time))

    return (df_bert, pd.DataFrame(time_list))'''

In [ ]:
'''id = df_pp['pair_id']
overall = df_pp['Overall']

pd.concat([id, overall], axis=1)'''

In [ ]:
'''from sklearn.model_selection import train_test_split

train_samples = []
valid_samples = []

index=0

X = df_pp[[f'doc1_pipeline{index}', f'doc2_pipeline{index}']]
y = df_pp['Overall']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2, random_state = SEED)

train_df = pd.concat([X_train, y_train], axis=1)
train_df = train_df.reset_index()
valid_df = pd.concat([X_valid, y_valid], axis=1)
valid_df = valid_df.reset_index()


for i in range(1, len(train_df)):
    inp_example = InputExample(texts=[train_df.iloc[i][f'doc1_pipeline{index}'], train_df.iloc[i][f'doc2_pipeline{index}']], label=train_df.iloc[i]['Overall'])
    train_samples.append(inp_example)

for j in range(1, len(valid_df)):
    inp_example = InputExample(texts=[valid_df.iloc[j][f'doc1_pipeline{index}'], valid_df.iloc[j][f'doc2_pipeline{index}']], label=valid_df.iloc[j]['Overall'])
    valid_samples.append(inp_example)'''

In [ ]:
import pandas as pd
import numpy as np
import time

from sklearn.metrics.pairwise import cosine_similarity

from scipy import spatial

from sentence_transformers import evaluation
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses
from torch.utils.data import DataLoader

from sklearn.model_selection import train_test_split

SEED = 42


###Bert###
def get_bert(model, doc1, doc2):
    
    data = [doc1, doc2]
    sentence_embeddings = model.encode(data)

    infer1 = sentence_embeddings[0]
    infer2 = sentence_embeddings[1]
    
    cos_similarity = 1 - spatial.distance.cosine(infer1, infer2) #de 0 a 1
    
    return cos_similarity

def apply_bert(df, len_pipeline, model, model_name, fine_tune = False):

    train_batch_size = 1
    num_epochs = 2
    model_save_path = 'output/fine_tune_bert-'+model_name

    time_list = []

    df_bert = pd.DataFrame()
    for index in range(len_pipeline):
        if fine_tune==True:

            train_samples = []
            valid_samples = []

            X = df[[f'doc1_pipeline{index}', f'doc2_pipeline{index}']]
            y = df['Overall']

            X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2, random_state = SEED)

            train_df = pd.concat([X_train, y_train], axis=1)
            train_df = train_df.reset_index()
            valid_df = pd.concat([X_valid, y_valid], axis=1)
            valid_df = valid_df.reset_index()


            ### testar sem tokenizar
            for i in range(1, len(train_df)):
                inp_example = InputExample(texts=[train_df.iloc[i][f'doc1_pipeline{index}'], train_df.iloc[i][f'doc2_pipeline{index}']], label=float(train_df.iloc[i]['Overall']))
                train_samples.append(inp_example)

            for j in range(1, len(valid_df)):
                inp_example = InputExample(texts=[valid_df.iloc[j][f'doc1_pipeline{index}'], valid_df.iloc[j][f'doc2_pipeline{index}']], label=float(valid_df.iloc[j]['Overall']))
                valid_samples.append(inp_example)

            train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)
            train_loss = losses.CosineSimilarityLoss(model=model)
            evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(valid_samples)#(sts_reader.get_examples('sts-dev.csv'))

            model.fit(train_objectives=[(train_dataloader, train_loss)],
                    evaluator=evaluator,
                    epochs=num_epochs,
                    evaluation_steps=1000,
                    #warmup_steps=warmup_steps, 100?
                    output_path=model_save_path
                    )

        ## ALTERAR: Avaliar no dataset de test
        start_time = time.time()
        df_bert[f'bert_{model_name}{index}'] = df.apply(lambda row: get_bert(model, " ".join(row[f'doc1_pipeline{index}']), " ".join(row[f'doc2_pipeline{index}'])), axis=1)
        time_list.append((f'bert_{model_name}{index}', time.time()-start_time))

    return (df_bert, pd.DataFrame(time_list))

In [ ]:
model_name = 'all-mpnet-base-v2'
#'multi-qa-mpnet-base-dot-v1'
#'all-distilroberta-v1'
#'all-MiniLM-L12-v2'
#'multi-qa-distilbert-cos-v1'

model = SentenceTransformer('sentence-transformers/'+model_name)

(df_bert, time_bert) = apply_bert(df_pp[:50], len(pre_processing_list), model, model_name, fine_tune=True)

df_results = df_results.join(df_bert)

In [ ]:
df_bert.to_csv('./resultados/'+model_name+'.csv', index = False)
time_bert.to_csv('./resultados/'+model_name+'.csv', index = False)

In [ ]:
df_results.corr()

## USE (Universal Sentence Encoder)

In [16]:
pre_processing_list = [
    {},
    {"no_url": True, "basic_processing": True}
    ]

df_pp = pp.pre_process_all(df_text, pre_processing_list)

In [17]:
from absl import logging

import tensorflow as tf

import tensorflow_hub as hub
import numpy as np
import os
import pandas as pd
import re
import math
import scipy

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [ ]:
df_pp.columns

In [25]:
doc1_name = 'doc1'
doc2_name = 'doc2'

sts_data = df_pp[['Overall', doc1_name, doc2_name]]#sts_dev

def run_sts_benchmark(batch):
  sts_encode1 = tf.nn.l2_normalize(embed(tf.constant(batch[doc1_name].tolist())), axis=1)
  sts_encode2 = tf.nn.l2_normalize(embed(tf.constant(batch[doc2_name].tolist())), axis=1)

  cosine_similarities = tf.reduce_sum(tf.multiply(sts_encode1, sts_encode2), axis=1)
  clip_cosine_similarities = tf.clip_by_value(cosine_similarities, -1.0, 1.0)
  scores = 1.0 - tf.acos(clip_cosine_similarities) / math.pi
  """Returns the similarity scores"""
  return scores

dev_scores = sts_data['Overall'].tolist()
scores = []
for batch in np.array_split(sts_data, 10):
  scores.extend(run_sts_benchmark(batch))

pearson_correlation = scipy.stats.pearsonr(scores, dev_scores) #scores= predicted cos_sim, dev_scores = Overall
print('Pearson correlation coefficient = {0}\np-value = {1}'.format(
    pearson_correlation[0], pearson_correlation[1]))

Pearson correlation coefficient = -0.7655629969170454
p-value = 8.087010431362859e-262


In [10]:
len(scores)

1357

## tf-idf

In [12]:
pre_processing_list = [
    {"tokenization": True},
    {"no_url": True, "basic_processing": True, "tokenization": True}
    ]


df_pp = pp.pre_process_all(df_text, pre_processing_list)

(df_tf_idf, time_tf_idf) = nlp.apply_tf_idf(df_pp, len(pre_processing_list))

In [ ]:
df_tf_idf.to_csv('./resultados/tf_idf.csv', index = False)
time_tf_idf.to_csv('./resultados/tf_idf_time.csv', index = False)

In [13]:
df_results = df_results.join(df_tf_idf)

In [14]:
df_results.head()

,pair_id,Overall,tf_idf0,tf_idf1
0,1484084337_1484110209,4.000000,0.117860,0.124465
1,1484396422_1483924666,3.666667,0.116576,0.111068
2,1484698254_1483758694,2.333333,0.312843,0.320284
3,1576314516_1576455088,2.000000,0.415904,0.398309
4,1484036253_1483894099,1.250000,0.551547,0.477166


In [15]:
df_results.corr()

,Overall,tf_idf0,tf_idf1
Overall,1.000000,-0.760434,-0.755391
tf_idf0,-0.760434,1.000000,0.997246
tf_idf1,-0.755391,0.997246,1.000000


## Bow

In [ ]:
(df_bow, time_bow) = nlp.apply_bow(df_pp, len(pre_processing_list))

In [ ]:
df_bow.to_csv('./resultados/bow.csv', index = False)
time_bow.to_csv('./resultados/bow_time.csv', index = False)

In [ ]:
df_results = df_results.join(df_bow)

In [ ]:
df_results.head(5)

In [ ]:
df_results.corr()

## countvectorizer (tfidf?)

In [ ]:
def calcular_y_pred(limite, distancia):

    y_pred = [1 if num >= limite else 0 for num in distancia]

    return y_pred


def montar_df_resultado(y_teste, y_pred, df_teste, nome):
    df_y = pd.DataFrame(
                        list(zip(
                                 y_teste, y_pred,
                                 df_teste["categoria"].to_list(),
                                 df_teste["titulo_1"].to_list(),
                                 df_teste["titulo_2"].to_list()
                                )
                       ), columns = ['match', 'pred', 'categoria', 'titulo_1', 'titulo_2'])

    return df_y


def salvar_distancia(y_teste, distancia, df_teste, nome):

    df_y = montar_df_resultado(y_teste, distancia, df_teste, nome)
    df_y.to_csv(f'Dados/Resultados/Cos/{nome}_distancia.csv', index = False)


def salvar_y_pred(y_teste, y_pred, df_teste, nome, limite):

    df_y = montar_df_resultado(y_teste, y_pred, df_teste, nome)
    df_y.to_csv(f'Dados/Resultados/Cos_{limite}/Resultado/{nome}_y.csv', index = False)


def salvar_relatorio(y_teste, y_pred, nome, tempo, limite):

    relatorio = classification_report(y_teste, y_pred, output_dict = True)
    df_relatorio = pd.DataFrame(relatorio).transpose()
    df_relatorio['modelo'] = nome
    df_relatorio['tempo'] = tempo

    df_relatorio.to_csv(f'Dados/Resultados/Cos_{limite}/RelatÃƒÂ³rio/{nome}_relatÃƒÂ³rio.csv', index = True)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

def formatar_entrada_bow(dados, mf = 1000):
    
    cv = CountVectorizer(
                         lowercase = True,
                         strip_accents = 'unicode',
                         max_features = mf
                        )

    cv.fit(dados)
    dados_transformados = cv.transform(dados).toarray()

    #X = matriz.fit_transform(dados).toarray()
    
    return cv, dados_transformados

def calcular_dis_cos(vetor_1, vetor_2):
                
    return (1 - spatial.distance.cosine(vetor_1, vetor_2))


In [ ]:
def pipeline_cos(df_teste, df_concat, tam_max):

    cv, titulo_bow = formatar_entrada_bow(df_concat, mf = tam_max)

    tam_df_concat = len(titulo_bow)/2
    tam_df_teste = df_teste.shape[0]

    if tam_df_concat == tam_df_teste:

        distancia = []
        for i in range( tam_df_teste ):
            distancia.append(calcular_dis_cos(titulo_bow[i], titulo_bow[i + tam_df_teste]))

        return distancia

    else:
        print ("TAMANHO ERRADO")
        return "ERRO"

In [ ]:
flag_cos = False
if flag_cos == True:

    for nome, df_teste in zip(arquivos, lista_df_teste):

        y_teste = df_teste["match"].to_list()

        # remoÃƒÂ§ÃƒÂ£o de pontuaÃƒÂ§ÃƒÂ£o e acentos
        ranking.fazer_pre_processamento(df_teste)

        # colocando os titulos em um dataframe com 1 coluna sÃƒÂ³
        df_concat = ranking.concatenar_titulos(df_teste)

        # calculando o tamanho mÃƒÂ¡ximo do tÃƒÂ­tulo
        tam_max = ranking.calcular_tam_max(df_concat)

        inicio_tempo = time.time()
        distancia = pipeline_cos(df_teste, df_concat, tam_max)
        final_tempo = time.time()

        tempo = final_tempo - inicio_tempo

        salvar_distancia(y_teste, distancia, df_teste, nome)

        for limite in VAR_COS:

            y_pred = calcular_y_pred(limite, distancia)

            salvar_y_pred(y_teste, y_pred, df_teste, nome, ranking.remove_pontuacao(str(limite)) )
            salvar_relatorio(y_teste, y_pred, nome, tempo, ranking.remove_pontuacao(str(limite)) )